In [1]:
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
from plotutils import plot_data, plot_surface
from SVM import kernel_svm_train, kernel_svm_predict
from sklearn import preprocessing


In [2]:
# Read the data using pandas
df = pd.read_csv('digit-recognition/digit-recognition/train.csv')
df2 = pd.read_csv('digit-recognition/digit-recognition/test.csv')

print(df.head(10))

#turn the training dataframe into numpy array 
label_data_train = df.to_numpy()

#column vector for the labels (38000,)
label_train = label_data_train[:,0] 

# (38000 x 784 matrix for the training data )
data_train = label_data_train[:, 1:] 
print(label_train.shape)
print(data_train.shape)

# Turn testing dataframe into numpy array 
label_data_test = df2.to_numpy()
label_test = label_data_test[:,0]
data_test = label_data_test[:,1:]

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      8       0       0       0       0       0       0       0       0   
1      8       0       0       0       0       0       0       0       0   
2      0       0       0       0       0       0       0       0       0   
3      5       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   
5      0       0       0       0       0       0       0       0       0   
6      3       0       0       0       0       0       0       0       0   
7      8       0       0       0       0       0       0       0       0   
8      2       0       0       0       0       0       0       0       0   
9      1       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1      

In [11]:
for i in range(10):
    train_indices = np.where(label_train==i)[0]
    test_indices = np.where(label_test==i)[0]

    masked_train_labels = np.full(label_train.shape, -1)
    masked_train_labels[train_indices] = 1

    masked_test_labels = np.full(label_test.shape, -1)
    masked_test_labels[test_indices] = 1

    best_params = {}
    best_params['kernel'] = 'rbf'
    best_params['C'] = 1
    best_params['sigma'] = .1

    split_train_data = np.split(data_train, 50)[0]
    split_train_labels = np.split(masked_train_labels, 50)[0]
    
    split_train_data_normalized = preprocessing.normalize(split_train_data, norm='l1')

    model_ones = kernel_svm_train(split_train_data_normalized.T, split_train_labels, best_params)
    predictions = kernel_svm_predict(data_test.T, model_ones)
    test_accuracy = np.mean(predictions[0] == masked_test_labels)
    print("Test accuracy for {0}: {1}".format(i, test_accuracy))

Test accuracy for 0: 0.9
Test accuracy for 1: 0.8885
Test accuracy for 2: 0.8925
Test accuracy for 3: 0.9075
Test accuracy for 4: 0.90275
Test accuracy for 5: 0.90525
Test accuracy for 6: 0.89675
Test accuracy for 7: 0.10225
Test accuracy for 8: 0.90425
Test accuracy for 9: 0.90475


In [10]:
from sklearn import preprocessing

for i in range(10):
    train_indices = np.where(label_train==i)[0]
    test_indices = np.where(label_test==i)[0]

    masked_train_labels = np.full(label_train.shape, -1)
    masked_train_labels[train_indices] = 1

    masked_test_labels = np.full(label_test.shape, -1)
    masked_test_labels[test_indices] = 1

    best_params = {}
    best_params['kernel'] = 'polynomial'
    best_params['C'] = 1
    best_params['sigma'] = .1
    best_params['order'] = 5

    split_train_data = np.split(data_train, 50)[0]
    split_train_labels = np.split(masked_train_labels, 50)[0]
    
    split_train_data_normalized = preprocessing.normalize(split_train_data, norm='l1')

    model_ones = kernel_svm_train(split_train_data_normalized.T, split_train_labels, best_params)
    predictions = kernel_svm_predict(data_test.T, model_ones)
    test_accuracy = np.mean(predictions[0] == masked_test_labels)
    print("Test accuracy for {0}: {1}".format(i, test_accuracy))

Test accuracy for 0: 0.9
Test accuracy for 1: 0.8885
Test accuracy for 2: 0.8925
Test accuracy for 3: 0.9075
Test accuracy for 4: 0.90275
Test accuracy for 5: 0.90525
Test accuracy for 6: 0.89675
Test accuracy for 7: 0.89775
Test accuracy for 8: 0.90425
Test accuracy for 9: 0.90475
